# 📘 Introducción

## ¿Qué es el polipropileno?

El **polipropileno (PP)** es un polímero termoplástico perteneciente a la familia de las poliolefinas. Se obtiene mediante la **polimerización del propileno**, un derivado del petróleo.  
Es uno de los plásticos más utilizados a nivel mundial debido a su bajo costo, versatilidad, ligereza y resistencia química.

Entre sus principales aplicaciones destacan:

- Empaques y envases  
- Industria automotriz  
- Textiles y fibras  
- Construcción  
- Dispositivos médicos  
- Productos del hogar  

Su combinación de precio accesible, resistencia y facilidad de moldeo lo convierten en un material indispensable en la manufactura moderna.

---

## ¿Por qué su precio es relevante?

El precio del polipropileno es un indicador económico importante, pues está influenciado por factores interconectados como:

- Precios del petróleo y gas natural  
- Oferta y demanda global  
- Situación de la industria petroquímica  
- Políticas comerciales y condiciones macroeconómicas  
- Paros técnicos o fallas en plantas de producción  

Su relevancia se debe a que el PP afecta directamente los **costos de producción** de múltiples industrias.  
Por ello, contar con **predicciones confiables del precio del PP** ayuda a:

- Planificar compras y presupuestos  
- Evaluar riesgos de precios  
- Estimar márgenes  
- Realizar análisis económicos de mercado  

---

## Breve revisión del dataset

Los datos fueron obtenidos mediante un método de scraping controlado desde TradingEconomics, tomando los valores históricos del precio spot del polipropileno.

El dataset contiene dos columnas principales:

- **Date** — Fecha de registro  
- **Price** — Precio del polipropileno por tonelada  

Ejemplo de datos:

| Date       | Price |
|------------|-------|
| 2024-11-20 | 7489  |
| 2024-11-21 | 7510  |
| 2024-11-22 | 7504  |
| 2024-11-25 | 7467  |
| 2024-11-26 | 7479  |

Características del dataset:

- Frecuencia: **Diaria**  
- Formato: **serie temporal**  
- Moneda: **USD por tonelada**  
- Datos limpios y sin valores nulos  
- Ordenados de forma cronológica  



---


In [5]:
#importar datos desde tampermokey
import pandas as pd
import requests

df = pd.DataFrame(requests.get("http://localhost:8765/pp").json())

df = df.rename(columns={"date": "Date", "value": "Price"})
df["Date"] = pd.to_datetime(df["Date"])
df.head()


KeyError: 'Date'

In [6]:
#imortar datos desde csv
import pandas as pd
from pandas import read_csv 
df = pd.DataFrame(read_csv("commodity_data.csv"))
df = df.rename(columns={"date": "Date", "value": "Price"})
df["Date"] = pd.to_datetime(df["Date"])
df.head()

,Date,Price
0,2024-11-21,7510
1,2024-11-22,7504
2,2024-11-25,7467
3,2024-11-26,7479
4,2024-11-27,7481


In [7]:
import pandas as pd
import plotly.express as px



fig = px.line(
    df,
    x="Date",
    y="Price",
    title="Precio del Polipropileno (PP) — Histórico",
    markers=True
)

fig.update_layout(
    xaxis_title="Fecha",
    yaxis_title="Precio (CNY/T)",
    hovermode="x unified",
    template="plotly_dark",
    title_font=dict(size=22),
    height=500
)



In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df["Price_scaled"] = scaler.fit_transform(df[["Price"]])
df.head()


,Date,Price,Price_scaled
0,2024-11-21,7510,1.000000
1,2024-11-22,7504,0.994614
2,2024-11-25,7467,0.961400
3,2024-11-26,7479,0.972172
4,2024-11-27,7481,0.973968


In [9]:
import numpy as np

data = df["Price_scaled"].values

train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data  = data[train_size:]


In [10]:
def create_windows(series, window_size=30):
    X, y = [], []
    for i in range(len(series) - window_size):
        X.append(series[i:i + window_size])
        y.append(series[i + window_size])
    return np.array(X), np.array(y)


In [11]:
WINDOW = 30  # puedes cambiarlo después

X_train, y_train = create_windows(train_data, WINDOW)
X_test,  y_test  = create_windows(test_data,  WINDOW)

# Ajustar dimensiones para LSTM/CNN
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test  = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

X_train.shape, X_test.shape


((163, 30, 1), (19, 30, 1))

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(WINDOW, 1)),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")
model.summary()

history = model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)


c:\Users\prisc\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,345 (114.63 KB)

 Trainable params: 29,345 (114.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 145ms/step - loss: 0.3894 - val_loss: 0.0177
Epoch 2/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0416 - val_loss: 0.0870
Epoch 3/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0313 - val_loss: 0.0345
Epoch 4/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0235 - val_loss: 0.0257
Epoch 5/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0181 - val_loss: 0.0386
Epoch 6/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0146 - val_loss: 0.0557
Epoch 7/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0184 - val_loss: 0.0462
Epoch 8/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0129 - val_loss: 0.0333
Epoch 9/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0141 - val_loss: 0.0369
Epoch 10/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0116 - val_loss: 0.0436
Epoch 11/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0116 - val_loss: 0.0401
Epoch 12/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0105 - val_loss: 0.0353


## MSE y MAPE

In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Predicción en test
y_pred_scaled = model.predict(X_test)

# Invertir escala
y_pred = scaler.inverse_transform(y_pred_scaled)
y_true = scaler.inverse_transform(y_test.reshape(-1, 1))

mse = mean_squared_error(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred) * 100

print("MSE:", mse)
print("MAPE (%)", mape)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 721ms/step
MSE: 26643.337152706965
MAPE (%) 2.3030054556688437


## Visualización real vs. Predicción

In [14]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    y=y_true.flatten(),
    mode="lines",
    name="Real (test)"
))

fig.add_trace(go.Scatter(
    y=y_pred.flatten(),
    mode="lines",
    name="Predicción LSTM (test)",
    line=dict(dash="dot")
))

fig.update_layout(
    title="Real vs Predicción — Conjunto de Prueba",
    xaxis_title="Índice (tiempo)",
    yaxis_title="Precio (CNY/T)",
    template="plotly_dark"
)

fig.show()


## Predicciones 7 días

In [15]:
n_future = 7
future_predictions = []

last_window = data[-WINDOW:]
current = last_window.reshape(1, WINDOW, 1)

for _ in range(n_future):
    next_scaled = model.predict(current)[0][0]
    future_predictions.append(next_scaled)
    current = np.append(current[:,1:,:], [[[next_scaled]]], axis=1)

future_real = scaler.inverse_transform(np.array(future_predictions).reshape(-1,1)).flatten()

future_dates = pd.date_range(df["Date"].iloc[-1], periods=n_future+1, freq="D")[1:]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


## Visualización próximas

In [16]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df["Date"],
    y=df["Price"],
    mode="lines",
    name="Histórico"
))

fig.add_trace(go.Scatter(
    x=future_dates,
    y=future_real,
    mode="lines+markers",
    name="Predicción futura"
))

fig.update_layout(
    title="Predicción de los próximos 7 días",
    xaxis_title="Fecha",
    yaxis_title="Precio",
    template="plotly_dark"
)

fig.show()


## Predicciones

In [18]:
future_table = pd.DataFrame({
    "Fecha futura": ["2025-11-24", "2025-11-25", "2025-11-26", "2025-11-27", "2025-11-28"],
    "Predicción ($/ton)": future_real[:5]
})

future_table


,Fecha futura,Predicción ($/ton)
0,2025-11-24,6555.865723
1,2025-11-25,6553.203125
2,2025-11-26,6554.112793
3,2025-11-27,6557.659180
4,2025-11-28,6563.233887
